# Visualizing prob-maps

In [1]:
import pickle
import itertools
import glob
import gc
import random
import sys

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from copy import copy, deepcopy
from tqdm.auto import tqdm, trange
from statsmodels.stats.proportion import proportion_confint

from pyecsca.ec.mult import *
from pyecsca.ec.countermeasures import *

from epare.config import all_configs, Config, MultIdent, CountermeasureIdent
from epare.prob_map import ProbMap
from epare.error_model import ErrorModel
from epare.filters import *

if sys.version_info >= (3, 14):
    from compression import zstd
else:
    from backports import zstd

%matplotlib ipympl

## Setup
Setup some plotting and the computations of prob-maps out of the small scalar data and divisors.

In [2]:
# mult_styles = ['-', '--', '-.', ':', (5, (10, 3)), (0, (5, 1)), (0, (3, 1, 1, 1, 1, 1)), (0, (3, 1, 1, 1)), (0, (1, 1)), (0, (3, 10, 1, 10))]
# mult_markers = [None, "o", "+", "*", "^", "s"]

majticks = np.arange(0, 1, 0.1)
minticks = np.arange(0, 1, 0.05)

## Divisors
The cell below contains some interesting divisors for distinguishing scalarmults.

In [3]:
from epare.divisors import divisor_map
for d, ds in divisor_map.items():
    print(f"{d:<27}", ds[:3], "...", ds[-1:])

small_primes                [3, 5, 7] ... [199]
medium_primes               [211, 223, 227] ... [397]
large_primes                [401, 409, 419] ... [997]
all_primes                  [3, 5, 7] ... [997]
all_integers                [1, 2, 3] ... [399]
all_even                    [2, 4, 6] ... [398]
all_odd                     [1, 3, 5] ... [399]
powers_of_2                 [2, 4, 8] ... [524288]
powers_of_2_large           [2, 4, 8] ... [57896044618658097711785492504343953926634992332820282019728792003956564819968]
powers_of_2_large_3         [6, 12, 24] ... [173688133855974293135356477513031861779904976998460846059186376011869694459904]
powers_of_2_large_p1        [3, 5, 9] ... [57896044618658097711785492504343953926634992332820282019728792003956564819969]
powers_of_2_large_m1        [1, 3, 7] ... [57896044618658097711785492504343953926634992332820282019728792003956564819967]
powers_of_2_large_pmautobus [1, 2, 3] ... [5789604461865809771178549250434395392663499233282028201972879200395

## Loading
Load the merged probability maps.

In [4]:
with zstd.open("merged.zpickle", "rb") as f:
    config_map = pickle.load(f)

## Plots

### 1. Error models
First, lets plot the effect different error models have on the error probability given the same multiplier and countermeasure.

In [ ]:
plot_configs = []
plot_divisors = divisor_map["small_primes"]

# Here are several useful filters when playing around with the data. We want to select a single multiplier and countereasure combination.
# See the filters module for more.
#   only_ltr_example
#   only_rtl_example
#   only_sliding_example
#   only_comb_example
#   single_layer_ctr
#   single_type_ctr
#   single_type_ctr_full

groups = {}
for config, probmap in config_map.items():
    if config.composition.klass == GroupScalarRandomization and single_layer_ctr(config) and only_rtl_example(config):
        plot_configs.append(config)
        pmap = deepcopy(probmap)
        pmap.narrow(plot_divisors)
        group = groups.setdefault(pmap.id(), [])
        group.append(config)
print(f"Plotting {len(plot_configs)} configs in {len(groups)} groups:")
inverse_groups = {}
for i, group in groups.items():
    for c in group:
        inverse_groups[c] = i
        print(c.error_model)
    print()

L = len(plot_divisors)
N = len(plot_mults)
x = list(range(L))

fig = plt.figure(figsize=(20, 10))
ax = plt.subplot(111)
colors = matplotlib.cm.tab10(range(8))
color_map = {
    "affine": 0,
    "affine,equal_multiples": 1,
    "affine,divides": 2,
    "affine,half_add": 3,
    "affine,divides,equal_multiples": 4,
    "affine,equal_multiples,half_add": 5,
    "affine,divides,half_add": 6,
    "affine,divides,equal_multiples,half_add": 7
}

style_map = {}
for i, config in enumerate(plot_configs):
    probmap = config_map[config]
    y_values = [probmap[l] for l in plot_divisors]
    # Use the same style for several entries that are fully overlapping (in the same group)
    group = inverse_groups[config]
    if group in style_map:
        style = style_map[group]
    else:
        style = dict(color=colors[color_map[",".join(sorted(config.error_model.checks))]],
                 marker="o" if config.error_model.precomp_to_affine else "",
                 linestyle="-" if config.error_model.check_condition == "all" else "--")
        style_map[group] = style
    ax.plot(x, y_values, **style, label=str(config.error_model))

ax.set_xlabel("divisor")
ax.set_ylabel("error probability")
ax.set_ylim((-0.05, 1.05))
ax.set_yticks(majticks)
ax.set_yticks(minticks, minor=True)
ax.set_xticks(x, plot_divisors, rotation=90)

ax.grid(axis="y", which="major", alpha=0.7)
ax.grid(axis="y", which="minor", alpha=0.3)
ax.grid(axis="x", alpha=0.7)
#plt.tight_layout()
box = ax.get_position()
ax.set_position([box.x0-0.08, box.y0, box.width, box.height+0.08])

ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

fig.savefig("error_models.pdf")
plt.close();

### 2. Countermeasures
Next up, we can have a look at how the countermeasure(s) used change the error probability, given a single multiplier and error model.

In [ ]:
plot_configs = []
plot_divisors = divisor_map["small_primes"]

# Here are several useful filters when playing around with the data. We want to select a single multiplier and error model.
# See the filters module for more.
#   only_ltr_example
#   only_rtl_example
#   only_sliding_example
#   only_comb_example
#   fixed_error_model
#   single_layer_ctr
#   single_type_ctr
#   single_type_ctr_full

groups = {}
for config, probmap in config_map.items():
    if fixed_error_model(config) and single_layer_ctr(config) and only_ltr_example(config):
        plot_configs.append(config)
        pmap = deepcopy(probmap)
        pmap.narrow(plot_divisors)
        group = groups.setdefault(pmap.id(), [])
        group.append(config)
print(f"Plotting {len(plot_configs)} configs in {len(groups)} groups:")
inverse_groups = {}
for i, group in groups.items():
    for c in group:
        inverse_groups[c] = i
        print(c)
    print()

L = len(plot_divisors)
N = len(plot_mults)
x = list(range(L))

fig = plt.figure(figsize=(20, 10))
ax = plt.subplot(111)
colors = matplotlib.cm.tab10(range(len(groups)))
color_map = {group: colors[i] for i, group in enumerate(groups)}

style_map = {}
for i, config in enumerate(plot_configs):
    probmap = config_map[config]
    y_values = [probmap[l] for l in plot_divisors]
    # Use the same style for several entries that are fully overlapping (in the same group)
    group = inverse_groups[config]
    if group in style_map:
        style = style_map[group]
    else:
        style = dict(color=color_map[group],
                 marker="o",
                 linestyle="-")
        style_map[group] = style
    ax.plot(x, y_values, **style, label=str(config.composition.klass.__name__))

ax.set_xlabel("divisor")
ax.set_ylabel("error probability")
ax.set_ylim((-0.05, 1.05))
ax.set_yticks(majticks)
ax.set_yticks(minticks, minor=True)
ax.set_xticks(x, plot_divisors, rotation=90)

ax.grid(axis="y", which="major", alpha=0.7)
ax.grid(axis="y", which="minor", alpha=0.3)
ax.grid(axis="x", alpha=0.7)
#plt.tight_layout()
box = ax.get_position()
ax.set_position([box.x0-0.08, box.y0, box.width, box.height+0.08])

ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

fig.savefig("countermeasures.pdf")
plt.close();

### 3. Multipliers (small primes)
We can also have a look at how different scalar multipliers influence the error probability.

In [ ]:
plot_configs = []
plot_divisors = divisor_map["small_primes"]

# Here are several useful filters when playing around with the data. We want to select a single countermeasure config and error_model
# See the filters module for more.
#   only_ltrs
#   only_rtls
#   only_slidingws
#   only_combs
#   no_combs
#   fixed_error_model
#   fixed_no_countermeasure
#   single_layer_ctr
#   single_type_ctr
#   single_type_ctr_full
#   has_gsr

groups = {}
for config, probmap in config_map.items():
    if fixed_error_model(config) and fixed_no_countermeasure(config) and no_combs(config):
        plot_configs.append(config)
        pmap = deepcopy(probmap)
        pmap.narrow(plot_divisors)
        group = groups.setdefault(pmap.id(), [])
        group.append(config)
print(f"Plotting {len(plot_configs)} configs in {len(groups)} groups:")
inverse_groups = {}
for i, group in groups.items():
    for c in group:
        inverse_groups[c] = i
        print(c.mult)
    print()

L = len(plot_divisors)
N = len(plot_mults)
x = list(range(L))

fig = plt.figure(figsize=(20, 10))
ax = plt.subplot(111)
colors = matplotlib.cm.tab20(range(12))
color_map = {
    LTRMultiplier: 0,
    RTLMultiplier: 1,
    CoronMultiplier: 2,
    BinaryNAFMultiplier: 3,
    WindowNAFMultiplier: 4,
    FixedWindowLTRMultiplier: 5,
    SlidingWindowMultiplier: 6,
    WindowBoothMultiplier: 7,
    SimpleLadderMultiplier: 8,
    BGMWMultiplier: 9,
    CombMultiplier: 10,
    FullPrecompMultiplier: 11
}

label_map = {}
for i, config in enumerate(plot_configs):
    probmap = config_map[config]
    y_values = [probmap[l] for l in plot_divisors]
    style = dict(color=colors[color_map[config.mult.klass]],
                 marker="o",
                 linestyle="-")
    if config.mult.klass in label_map:
        label = "__nolegend__"
    else:
        label = config.mult.klass.__name__
        label_map[config.mult.klass] = label
    ax.plot(x, y_values, **style, label=label)

ax.set_xlabel("divisor")
ax.set_ylabel("error probability")
ax.set_ylim((-0.05, 1.05))
ax.set_yticks(majticks)
ax.set_yticks(minticks, minor=True)
ax.set_xticks(x, plot_divisors, rotation=90)

ax.grid(axis="y", which="major", alpha=0.7)
ax.grid(axis="y", which="minor", alpha=0.3)
ax.grid(axis="x", alpha=0.7)
#plt.tight_layout()
box = ax.get_position()
ax.set_position([box.x0-0.08, box.y0, box.width, box.height+0.08])

ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

fig.savefig("multipliers_small_primes.pdf")
plt.close();

### 4. Multiplier options (small primes)
Scalar multipliers are often parametrizable, we can examine how parameters like "width" in a window-based multiplier influence the error rate, given a single error model and no countermeasures.

In [ ]:
plot_configs = []
plot_divisors = divisor_map["small_primes"]

# Here are several useful filters when playing around with the data. We want to select a single countermeasure config and error_model
# See the filters module for more.
#   only_ltrs
#   only_rtls
#   only_slidingws
#   only_combs
#   no_combs
#   fixed_error_model
#   fixed_no_countermeasure
#   single_layer_ctr
#   single_type_ctr
#   single_type_ctr_full
#   has_gsr


groups = {}
for config, probmap in config_map.items():
    if fixed_error_model(config) and fixed_no_countermeasure(config) and only_slidingws(config):
        plot_configs.append(config)
        pmap = deepcopy(probmap)
        pmap.narrow(plot_divisors)
        group = groups.setdefault(pmap.id(), [])
        group.append(config)
print(f"Plotting {len(plot_configs)} configs in {len(groups)} groups:")
inverse_groups = {}
for i, group in groups.items():
    for c in group:
        inverse_groups[c] = i
        print(c.mult)
    print()

L = len(plot_divisors)
N = len(plot_mults)
x = list(range(L))

fig = plt.figure(figsize=(20, 10))
ax = plt.subplot(111)
colors = matplotlib.cm.tab10(range(len(groups)))
color_map = {group: colors[i] for i, group in enumerate(groups)}

label_map = {}
for i, config in enumerate(plot_configs):
    probmap = config_map[config]
    y_values = [probmap[l] for l in plot_divisors]
    # Use the same style for several entries that are fully overlapping (in the same group)
    group = inverse_groups[config]
    if group in style_map:
        style = style_map[group]
    else:
        style = dict(color=color_map[group],
                 marker="o",
                 linestyle="-")
        style_map[group] = style
    ax.plot(x, y_values, **style, label=str(config.mult))

ax.set_xlabel("divisor")
ax.set_ylabel("error probability")
ax.set_ylim((-0.05, 1.05))
ax.set_yticks(majticks)
ax.set_yticks(minticks, minor=True)
ax.set_xticks(x, plot_divisors, rotation=90)

ax.grid(axis="y", which="major", alpha=0.7)
ax.grid(axis="y", which="minor", alpha=0.3)
ax.grid(axis="x", alpha=0.7)
#plt.tight_layout()
box = ax.get_position()
ax.set_position([box.x0-0.08, box.y0, box.width, box.height+0.08])

ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

fig.savefig("multiplier_options_small_primes.pdf")
plt.close();

## Misc (broken)

In [ ]:
for divisor_name in divisor_map:
    plot_mults = list(filter(lambda mult: mult in distributions_mults, all_mults_with_ctr))
    print(divisor_name, "allmults")
    plot_divisors = sorted(divisor_map[divisor_name])
    L = len(plot_divisors)
    N = len(plot_mults)
    x = list(range(L))
    
    fig = plt.figure(figsize=(L/4+10, 26))
    ax = plt.subplot(111)
    
    vals = np.zeros((N, L))
    n_samples = 0
    for i, mult in enumerate(plot_mults):
        probmap = distributions_mults[mult]
        y_values = [probmap[l] for l in plot_divisors]
        vals[i,] = y_values
        ax.plot(x, y_values,
                color=colors[mult],
                linestyle=styles[mult],
                marker=markers[mult],
                label=str(mult) if mult.countermeasure is None else "_nolegend_")
        if showci:
            cis = [conf_interval(p, probmap.samples) for p in y_values]
            ci_low = [ci[0] for ci in cis]
            ci_high = [ci[1] for ci in cis]
            ax.fill_between(x, ci_low, ci_high, color="black", alpha=0.1)
        n_samples += probmap.samples
    
    ax.set_title(f"{divisor_name} ({kind})\nSamples(avg): " + str(n_samples//N))
    
    #var = np.var(vals, axis=0)
    #ax.plot(x, var / np.max(var), label="cross-mult variance (normalized)", ls="--", lw=2, color="black")
    
    ax.set_xlabel("divisors")
    ax.set_ylabel("error probability")
    ax.set_yticks(majticks)
    ax.set_yticks(minticks, minor=True)
    ax.set_xticks(x, plot_divisors, rotation=90)
    
    ax.grid(axis="y", which="major", alpha=0.7)
    ax.grid(axis="y", which="minor", alpha=0.3)
    ax.grid(axis="x", alpha=0.7)
    plt.tight_layout()
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])
    
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    fig.savefig(f"graphs/{kind}-kind/{divisor_name}-allmults{'+ci' if showci else ''}.pdf")
    plt.close()